In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

In [2]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model

mod_RPE_PR = read_sbml_model(Path().cwd() / "models" / "mod_RPE_PR_old.xml")
mod_Human1_Human1 = read_sbml_model(Path().cwd()  / "models" / "mod_Human1_Human1.xml")
mod_Recon3D_Recon3D = read_sbml_model(Path().cwd()  / "models" / "mod_Recon3D_Recon3D.xml")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


KeyboardInterrupt: 

In [ ]:
# modify models

# set objective function (PR ATP DM)
mod_RPE_PR.objective = 'MAR03964_PR'
mod_Human1_Human1.objective = 'MAR03964_PR'
mod_Recon3D_Recon3D.objective = 'DM_atp_c__PR'
mitocore.objective = 'OF_ATP_MitoCore'


# close PR exchange reactions, open RPE exchange upper boundaries
from src.modify_model import close_PR_EX, open_RPE_EX_ub
models = [mod_RPE_PR, mod_Human1_Human1, mod_Recon3D_Recon3D]
for m in models:
    m = close_PR_EX(m)
    m = open_RPE_EX_ub(m)

In [4]:
mod_RPE_PR

Name,HumanGEM_HumanGEM
Memory address,21c8689b100
Number of metabolites,10436
Number of reactions,14282
Number of genes,2268
Number of groups,142
Objective expression,1.0*MAR03964_PR - 1.0*MAR03964_PR_reverse_e8402
Compartments,"c_RPE, r_RPE, e_RPE, m_RPE, x_RPE, g_RPE, l_RPE, n_RPE, i_RPE, e_RPE_PR, c_PR, x_PR, m_PR, g_PR, r_PR, l_PR, i_PR"


In [ ]:
# set simulation parameters

# exchange lower bounds (max flux into the model from outside world)
EX_o2_e = [-10] # oxygen
EX_glc_D = [-10] # glucose

# rxn IDs for Human1 models and Recon3Dmodel
o2_ID = ['MAR09048_RPE','MAR09048_RPE','EX_o2s[e]_RPE']
glc_ID = ['MAR09034_RPE','MAR09034_RPE','EX_glc_D[e]_RPE']
ATP_RPE_ID = ['MAR03964_RPE','MAR03964_RPE','DM_atp_c__RPE']

In [ ]:
models[0]

In [ ]:
from src.analysis import FVA_FBA_analysis
from datetime import datetime
from src.get_info import make_rxn_df
import pandas as pd 

# make df with rxns in model
datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')    
results = dict()
for m in models:
    df_collated = pd.DataFrame()
    i = 0
    with m as model:
        rxns = make_rxn_df(model)
        for EX_o2_e_i in EX_o2_e:
            model.reactions.get_by_id(o2_ID[i]).lower_bound = EX_o2_e_i
            for EX_glc_D_i in EX_glc_D:
                model.reactions.get_by_id(glc_ID[i]).lower_bound = EX_glc_D_i
                df_results = FVA_FBA_analysis(model,ATP_RPE_ID[i])
                df_results.columns = df_results.columns + '_O2_' + str(EX_o2_e_i) + '_GLC_' + str(EX_glc_D_i)
                df_collated = pd.concat([df_collated, df_results], axis=1)           
        df_collated = pd.merge(rxns, df_collated, left_index=True, right_index=True)   
        sheetname = model.id
        with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
            df_collated.to_excel(writer, sheet_name = sheetname) 
        results[i] = df_collated
    i = i + 1

In [ ]:
models[0]

In [ ]:
models[1].reactions.get_by_id('MAR09048_RPE')

In [ ]:
stop

In [ ]:
from src.analysis import FVA_FBA_analysis
from datetime import datetime
import pandas as pd

for m in models:
    df_collated = pd.DataFrame()
    with m as model:
        i = 0
        filename = m.id + '.xlsx'
        for EX_o2_e_i in EX_o2_e:
            model.reactions.get_by_id(o2_ID[i]).lower_bound = EX_o2_e_i
            for EX_glc_D_i in EX_glc_D:
                model.reactions.get_by_id(glc_ID[i]).lower_bound = EX_glc_D_i
                df_results = FVA_FBA_analysis(model,ATP_RPE_ID[i])
                df_results.columns = df_results.columns + '_O2_' + str(EX_o2_e_i) + '_GLC_' + str(EX_glc_D_i)
                df_collated = pd.merge(df_collated, df_results, left_index=True, right_index=True)       
        sheetname = model.id
        datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')
        with pd.ExcelWriter('results_' + datestr + '.xlsx') as writer:  
            df_collated.to_excel(writer, sheet_name = sheetname)
                
        i = i + 1

In [ ]:
    from src.analysis import FVA_FBA_analysis
    import pandas as pd

    with mod_RPE_PR as model:
        i = 0
        filename = m.id + '.xlsx'
        for EX_o2_e_i in EX_o2_e:
            model.reactions.get_by_id(o2_ID[i]).lower_bound = EX_o2_e_i
            for EX_glc_D_i in EX_glc_D:
                model.reactions.get_by_id(glc_ID[i]).lower_bound = EX_glc_D_i
                df_results = FVA_FBA_analysis(model,ATP_RPE_ID[i])
                sheetname = 'O2_' + str(EX_o2_e_i) + '_GLC_' + str(EX_glc_D_i)
                with pd.ExcelWriter(filename) as writer:  
                    df_results.to_excel(writer, sheet_name = sheetname)
        i = i + 1

    df_results

In [ ]:
from src.analysis import FVA_FBA_analysis

for m in models:
    with m as model:
        i = 0
        filename = m.id + '.xlsx'
        for EX_o2_e_i in EX_o2_e:
            model.reactions.get_by_id(o2_ID[i]).lower_bound = EX_o2_e_i
            for EX_glc_D_i in EX_glc_D:
                model.reactions.get_by_id(glc_ID[i]).lower_bound = EX_glc_D_i
                df_results = FVA_FBA_analysis(model,ATP_RPE_ID[i])
                sheetname = 'O2_' str(EX_o2_e_i) + '_GLC_' + str(EX_glc_D_i)
                with pd.ExcelWriter(filename) as writer:  
                    df_results.to_excel(writer, sheet_name = sheetname)
        i = i + 1

In [ ]:
# SET EXCHANGE BOUNDS
# 'id' col with IDs and an 'upper_bound' col with bounds
def set_bounds(model,bounds):
    
    # open RPE exchange reactions
    for i in range(len(bounds)): 
        try:
            model.reactions.get_by_id(bounds['id'][i]+'_RPE').bounds = (-bounds['upper_bound'][i],1000)
        except KeyError:
            model.reactions.get_by_id(bounds['vmh_id'][i]+'_RPE').bounds = (-bounds['upper_bound'][i],1000)
    
    # close PR exchange reactions  
    rxns = [r for r in model.reactions if '_PR' in r.id if len(r.products) == 0]
    for r in rxns:
        r.bounds=(0,0)
    return model

import pandas as pd  
bounds = pd.read_excel(Path().cwd() / 'rxn_bounds' / 'R3D301_EX_rxns.xlsx')
bounds = bounds[bounds['upper_bound']>0] # select entries with non-zero upper bounds

mod_RPE_PR = set_bounds(mod_RPE_PR,bounds)
mod_Human1_Human1 = set_bounds(mod_Human1_Human1,bounds)
mod_Recon3D_Recon3D  = set_bounds(mod_Recon3D_Recon3D,bounds)

In [ ]:
# perform FVA and FBA on a model setting RPE_ATP demand to fixed values

def FVA_FBA_analysis(model,DM_atp_c__PR_rxn_id):
    from cobra.flux_analysis import flux_variability_analysis
    import pandas as pd
        
    # make df with rxns in model
    rxns = make_rxn_df(model)
    
    df_FVA_full = pd.DataFrame()
    df_FBA_fluxes = pd.DataFrame()
    df_FBA_costs = pd.DataFrame()
    df_pFBA_fluxes = pd.DataFrame()
    df_pFBA_costs = pd.DataFrame()
    RPE_ATP = [0]

    for x in RPE_ATP:
        model.reactions.get_by_id(DM_atp_c__PR_rxn_id).bounds=(x,x) # set bounds

        fba_results = model.optimize() # traditional FBA
        fba = fba_results.to_frame() # df FBA results
        fba.columns = fba.columns + '_' + str(x) # add RPE_ATP to FBA column names
        df_FBA_fluxes = pd.concat([df_FBA_fluxes, fba.iloc[:,0]], axis=1) # fill df flux results
        df_FBA_costs = pd.concat([df_FBA_costs, fba.iloc[:,1]], axis=1) # fill df reduced costs

        fva_full = flux_variability_analysis(model, loopless=False) # FVA 
        fva_full.columns = fva_full.columns + '_' + str(x) # add RPE_ATP to FVA column names
        df_FVA_full = pd.concat([df_FVA_full, fva_full], axis=1) # fill df FVA results

        df_l = [df_FVA_full, df_FBA_fluxes, df_FBA_costs]
        df_results = rxns
    for df in df_l:
        df_results = pd.merge(df_results, df, left_index=True, right_index=True) 
    # fix column order
    min_max_flux = [i for i in list(df_results.columns) if 'min' in i] + \
                [i for i in list(df_results.columns) if 'max' in i] + \
                [i for i in list(df_results.columns) if 'flux' in i] 

    df_results = df_results.reindex([i for i in list(df_results.columns) if i not in min_max_flux] + min_max_flux, axis = 1)
    return df_results

mod_RPE_PR_results = FVA_FBA_analysis(mod_RPE_PR,'MAR03964_RPE')
#mod_Human1_Human1_results = FVA_FBA_analysis(mod_Human1_Human1,'MAR03964_RPE')
#mod_Recon3D_Recon3D_results = FVA_FBA_analysis(mod_Recon3D_Recon3D,'DM_atp_c__RPE')

In [ ]:
df_compact = df[['rxn_ID','subsystem','name','reaction','lb','ub'] + \
                [i for i in list(df.columns) if 'min' in i] + \
                [i for i in list(df.columns) if 'max' in i] + \
                [i for i in list(df.columns) if 'flux' in i] ]

m1 = [(df_compact['lb']==0) & (df_compact['ub']==0)] # lb=0 & ub=0 mask
zero_FBA = df[[i for i in list(df.columns) if 'flux' in i]]<10**-12
m2 = zero_FBA.all(axis=1) # lb=0 & ub=0 mask

In [ ]:
zero_FBA = df[[i for i in list(df.columns) if 'flux' in i]]<10**-12
df_compact[zero_FBA.all(axis=1)].to_clipboard()

In [ ]:
df[df['subsystem'] == 'Exchange/demand reactions']

In [ ]:
# post-processing (sorting) and saving of the results

def results_to_excel(df, path):
    # Write to Multiple Sheets
    with pd.ExcelWriter(path) as writer:
        df.to_excel(writer, sheet_name='full')
        df[df['subsystem'] == 'Exchange/demand reactions'].to_excel(writer, sheet_name='Exchange/demand reactions')
        
from datetime import datetime
datestr = datetime.strftime(datetime.now(), '%H%M_%d-%m-%Y')
filename = '' + datestr + '.xlsx'
results_to_excel(mod_RPE_PR_results,Path().cwd() / 'results' / filename)

In [ ]:
def add_compartment2rxns(mod):
    rxn_list = []
    met_list = [m.id for m in mod.metabolites]
    for r in mod.reactions:
        rxn = " ".join([mod.metabolites.get_by_id(x).name+'['+mod.metabolites.get_by_id(x).compartment+']'\
         if x in met_list else x for x in r.reaction.split()])
        rxn_list.append(rxn)
    return rxn_list

rxn_list_mod_RPE_PR = add_compartment2rxns(mod_RPE_PR)
rxn_list_mod_RPE_PR_no_CORDA = add_compartment2rxns(mod_RPE_PR_no_CORDA)

In [ ]:
mod_RPE_PR_results2[mod_RPE_PR_results2['subsystem'] == 'Exchange/demand reactions'].to_clipboard()

In [ ]:
mod_RPE_PR_results2[mod_RPE_PR_results2['cell'] =='RPE_PR_interface'].to_clipboard()

In [ ]:
# interface reactions
df1 = mod_RPE_PR_no_CORDA_results2.loc[[l for l in list(mod_RPE_PR_no_CORDA_results2.index) if 'RPE_PR' in l]]
df2 =  mod_RPE_PR_no_CORDA_results2.loc[[l for l in list(mod_RPE_PR_no_CORDA_results2.index) if 'PR_RPE' in l]]
df12 = pd.concat([df1,df2], axis=0)
df12.to_clipboard()

In [ ]:
from cobra.flux_analysis.loopless import add_loopless
mod_RPE_PR_ll = mod_RPE_PR.copy()
add_loopless(mod_RPE_PR_ll)

In [ ]:
# select non-zero FBA fluxes
s_flux = s.loc[:,[c for c in list(s.columns) if 'fluxes' in c]]
s_flux = s_flux.loc[(s_flux!=0).any(axis=1)]
s_flux.to_clipboard(excel=True, sep=None)

# RXNS
rxns = pd.DataFrame([[r.id,r.id.split('_')[1].replace('eRPE','RPE_PR_interface'),r.lower_bound,r.upper_bound, r.name,r.subsystem,\
                      r.reaction,r.build_reaction_string(use_metabolite_names = True),r.gpr]\
                      for r in mod_RPE_PR.reactions],index=[r.id for r in mod_RPE_PR.reactions],\
                    columns=['Human1.reaction','cell','lb','ub','name','subsystem',\
                             'reaction (IDs)','reaction (names)','GPR'])

fluxes_fba = pd.merge(rxns.copy(),s_flux,left_index=True, right_index=True) 
fluxes_fba.to_clipboard(excel=True, sep=None)
fluxes_fba